# 데이터프레임 다루기
- 데이터프레임 합치기
 - concat, append, join, merge
- 데이터프레임 분해하기
 - groupby
- pivot_table

## import

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## concat
- 테이블을 합치는 가장 기본적인 방법으로, 행이나 열의 크기가 같을 때 주로 사용한다
- 열방향(가로) 또는 행방향(세로)으로 합칠 수 있다. 즉, 컬럼을 늘리거나 샘플을 늘릴 때

In [41]:
data1 = {'city': ['서울', '부산', '대구', '대전', '광주'],
        'population': [990, 350, 250, 154, 150],
        'temp': [13, 16, 14, 13, 15]}

df1 = pd.DataFrame(data1)
df1

city  population  temp
0   서울         990    13
1   부산         350    16
2   대구         250    14
3   대전         154    13
4   광주         150    15

### 가로로 합치는 경우
- 인덱스를 기준으로 합친다

In [42]:
# 샘플 수가 다른 데이터 생성
data2 = {'city': ['서울', '부산', '대구', '대전'],
        'cars': [300, 140, 120, 70]}

# 샘플 수가 같은 경우
# data2 = {'city': ['서울', '부산', '대구', '대전', '광주'],
#         'cars': [300, 140, 120, 70, 50]}

df2 = pd.DataFrame(data2)
df2

city  cars
0   서울   300
1   부산   140
2   대구   120
3   대전    70

In [43]:
# 가로로 붙이기 (인덱스가 같은 행을 붙인다)
pd.concat((df1, df2), axis=1)

city  population  temp city   cars
0   서울         990    13   서울  300.0
1   부산         350    16   부산  140.0
2   대구         250    14   대구  120.0
3   대전         154    13   대전   70.0
4   광주         150    15  NaN    NaN

- (주의) 동일한 컬럼명(city)이 있으면 두 개 생긴다

### 세로로 합치는 경우
- 즉, 샘플을 합칠 때

In [44]:
# 열(컬럼)이 같은 데이터 생성
data3 = {'city': ['인천', '울산'],
        'population': [290, 120],
        'temp': [12.7, 15.5]}
df3 = pd.DataFrame(data3)
df3

city  population  temp
0   인천         290  12.7
1   울산         120  15.5

In [45]:
# 기존의 인덱스를 유지하면서 합치는 경우
pd.concat((df1, df3))

city  population  temp
0   서울         990  13.0
1   부산         350  16.0
2   대구         250  14.0
3   대전         154  13.0
4   광주         150  15.0
0   인천         290  12.7
1   울산         120  15.5

In [46]:
# 기존의 인덱스를 무시하고 인덱스를 새로 배정하는 경우
pd.concat((df1, df3), ignore_index=True)

city  population  temp
0   서울         990  13.0
1   부산         350  16.0
2   대구         250  14.0
3   대전         154  13.0
4   광주         150  15.0
5   인천         290  12.7
6   울산         120  15.5

## append
- 세로방향으로 샘플을 더할 때는 concat 대신에 append를 사용할 수 있다

In [47]:
df1.append(df3)

city  population  temp
0   서울         990  13.0
1   부산         350  16.0
2   대구         250  14.0
3   대전         154  13.0
4   광주         150  15.0
0   인천         290  12.7
1   울산         120  15.5

## join
- 행의 크기가 일치하지 않는 테이블을 가로 방향으로 붙이는 방법
- 기본적으로 인덱스를 기준으로 테이블을 붙인다

In [48]:
# df1에서 city 컬럼을 인덱스로 지정
df1 = pd.DataFrame(data1).set_index('city')
df1

population  temp
city                  
서울           990    13
부산           350    16
대구           250    14
대전           154    13
광주           150    15

In [49]:
# 인덱스와 컬럼 내용이 df1과 다른 새로운 데이터프레임 생성
data4 = {'city':['대전', '광주','인천', '울산'],
         'sale_1': [11, 21, 31, 41],
         'sale_2': [12, 22, 32, 42]
        }

df4 = pd.DataFrame(data4).set_index('city')
df4

sale_1  sale_2
city                
대전        11      12
광주        21      22
인천        31      32
울산        41      42

- 붙이는 두 데이터프레임의 인덱스가 일부만 겹칠 때

In [50]:
# 디폴트로 좌측 데이터프레임의 인덱스를 기준으로 붙이며 인덱스가 빈 곳에는 nan이 들어간다
df1.join(df4)

population  temp  sale_1  sale_2
city                                  
서울           990    13     NaN     NaN
부산           350    16     NaN     NaN
대구           250    14     NaN     NaN
대전           154    13    11.0    12.0
광주           150    15    21.0    22.0

In [51]:
# 위의 디폴트 동작과 같다 (좌측 기준)
df1.join(df4, how='left')

population  temp  sale_1  sale_2
city                                  
서울           990    13     NaN     NaN
부산           350    16     NaN     NaN
대구           250    14     NaN     NaN
대전           154    13    11.0    12.0
광주           150    15    21.0    22.0

In [52]:
# 우측 데이터프레임의 인덱스를 기준으로 붙이는 경우
df1.join(df4, how='right')

population  temp  sale_1  sale_2
city                                  
대전         154.0  13.0      11      12
광주         150.0  15.0      21      22
인천           NaN   NaN      31      32
울산           NaN   NaN      41      42

In [53]:
# 좌우 데이터프레임 인덱스 전체를 대상으로 붙이는 경우 
df1.join(df4, how='outer')

population  temp  sale_1  sale_2
city                                  
광주         150.0  15.0    21.0    22.0
대구         250.0  14.0     NaN     NaN
대전         154.0  13.0    11.0    12.0
부산         350.0  16.0     NaN     NaN
서울         990.0  13.0     NaN     NaN
울산           NaN   NaN    41.0    42.0
인천           NaN   NaN    31.0    32.0

- join에서 두 데이터프레임에 같은 컬럼명이 있으면 오류 발생
 - 미리 한 쪽의 컬럼명을 바꾸고 join해야 한다

## (연습) 좌우 데이터프레임 인덱스 중 공통부분만을 대상으로 붙이시오
- (힌트) 'inner" 옵션을 사용한다

In [89]:
# (풀이)



## merge
- 특정 열(컬럼)을 기준으로 데이터프레임 합치기
- 즉, 인덱스를 사용하지 않는다

In [55]:
# city를 인덱스로 사용하지 않는 데이터프레임 생성
df1 = pd.DataFrame(data1)
df1

city  population  temp
0   서울         990    13
1   부산         350    16
2   대구         250    14
3   대전         154    13
4   광주         150    15

In [56]:
# city를 인덱스로 사용하지 않는 데이터프레임 생성
df4 = pd.DataFrame(data4)
df4

city  sale_1  sale_2
0   대전      11      12
1   광주      21      22
2   인천      31      32
3   울산      41      42

In [57]:
# 특정 컬럼 city를 기준으로 합친다
df1.merge(df4, on='city', how='inner')

city  population  temp  sale_1  sale_2
0   대전         154    13      11      12
1   광주         150    15      21      22

In [58]:
# 위의 옵션은 디폴트 통작이다 (on과 how를 지정하지 않는 경우)
df1.merge(df4)

city  population  temp  sale_1  sale_2
0   대전         154    13      11      12
1   광주         150    15      21      22

In [59]:
# 좌측 데이터프레임의 컬럼 city를 기준으로 합치는 경우
df1.merge(df4, on='city', how='left')

city  population  temp  sale_1  sale_2
0   서울         990    13     NaN     NaN
1   부산         350    16     NaN     NaN
2   대구         250    14     NaN     NaN
3   대전         154    13    11.0    12.0
4   광주         150    15    21.0    22.0

In [60]:
# 우측 데이터프레임의 컬럼 city를 기준으로 합치는 경우
df1.merge(df4, on='city', how='right')

city  population  temp  sale_1  sale_2
0   대전       154.0  13.0      11      12
1   광주       150.0  15.0      21      22
2   인천         NaN   NaN      31      32
3   울산         NaN   NaN      41      42

## (연습) 두 데이터프레임의 컬럼 city를 기준으로 모든 행을 합치시오
- (힌트) outer 옵션을 사용한다

In [90]:
# (풀이)



# 테이블 합치기 예

In [62]:
# 전자제품 미국 수출 데이터, logistic은 운송 방법
x1 = {'item': ['eCar', 'eCar', 'TV',  'TV', 'Refr','Refr'],
        'logistic': ['ship', 'air', 'ship', 'air', 'ship', 'air'],
        '2019': [19, 27, 24, 13, 35, 23],
        '2020': [21, 102, 34, 87, 45, 30]}
dx1 = pd.DataFrame(x1)
dx1['region']='US'
dx1

item logistic  2019  2020 region
0  eCar     ship    19    21     US
1  eCar      air    27   102     US
2    TV     ship    24    34     US
3    TV      air    13    87     US
4  Refr     ship    35    45     US
5  Refr      air    23    30     US

In [63]:
# 중국 수출 데이터, logistic은 운송 방법
x2 = {'item': [ 'Refr', 'Refr', 'TV', 'TV', 'eCar', 'eCar'],
        'logistic': ['ship', 'air', 'ship', 'air', 'ship', 'air'],
        '2019': [29, 56, 45, 56, 48, 65],
        '2020': [180, 42, 56, 89, 98, 79]}
dx2 = pd.DataFrame(x2)
dx2['region']='China'
dx2

item logistic  2019  2020 region
0  Refr     ship    29   180  China
1  Refr      air    56    42  China
2    TV     ship    45    56  China
3    TV      air    56    89  China
4  eCar     ship    48    98  China
5  eCar      air    65    79  China

- 샘플 합치기, concat 사용

In [64]:
# 컬럼이 일치하므로 concat으로 샘플을 합칠 수 있다 (append도 같은 동작을 한다)
df = pd.concat([dx1,dx2], ignore_index=True)
# df = dx1.append(dx2, ignore_index=True)
df

item logistic  2019  2020 region
0   eCar     ship    19    21     US
1   eCar      air    27   102     US
2     TV     ship    24    34     US
3     TV      air    13    87     US
4   Refr     ship    35    45     US
5   Refr      air    23    30     US
6   Refr     ship    29   180  China
7   Refr      air    56    42  China
8     TV     ship    45    56  China
9     TV      air    56    89  China
10  eCar     ship    48    98  China
11  eCar      air    65    79  China

## 필터링
- 특정 조건을 만족하는 데이터를 추출하는 것

In [65]:
# 지역 필터링
df[df['region']=='US']

item logistic  2019  2020 region
0  eCar     ship    19    21     US
1  eCar      air    27   102     US
2    TV     ship    24    34     US
3    TV      air    13    87     US
4  Refr     ship    35    45     US
5  Refr      air    23    30     US

In [66]:
# 상품 필터링
df[df['item']=='Refr']

item logistic  2019  2020 region
4  Refr     ship    35    45     US
5  Refr      air    23    30     US
6  Refr     ship    29   180  China
7  Refr      air    56    42  China

In [67]:
# 두 가지 상품 필터링
df[df['item'].isin(['TV','eCar'])]

item logistic  2019  2020 region
0   eCar     ship    19    21     US
1   eCar      air    27   102     US
2     TV     ship    24    34     US
3     TV      air    13    87     US
8     TV     ship    45    56  China
9     TV      air    56    89  China
10  eCar     ship    48    98  China
11  eCar      air    65    79  China

In [68]:
# 두가지 조건을 만족하는 필터링
df[(df['region']=='China') & (df['item']=='Refr')]

item logistic  2019  2020 region
6  Refr     ship    29   180  China
7  Refr      air    56    42  China

## 참조 테이블과 합치기
- 상품별 가격표를 합치는 방법
- merge 사용

In [69]:
# 가격표
price_list = {'item': [ 'Refr', 'eCar', 'TV'],
        'price': [200, 1000, 100]}
df_price = pd.DataFrame(price_list)
df_price

item  price
0  Refr    200
1  eCar   1000
2    TV    100

In [70]:
# 가격표를 반영한(합친) 데이터프레임
# item 컬럼을 기준으로 합친다
df_all = df.merge(df_price, on="item")
df_all

item logistic  2019  2020 region  price
0   eCar     ship    19    21     US   1000
1   eCar      air    27   102     US   1000
2   eCar     ship    48    98  China   1000
3   eCar      air    65    79  China   1000
4     TV     ship    24    34     US    100
5     TV      air    13    87     US    100
6     TV     ship    45    56  China    100
7     TV      air    56    89  China    100
8   Refr     ship    35    45     US    200
9   Refr      air    23    30     US    200
10  Refr     ship    29   180  China    200
11  Refr      air    56    42  China    200

## (연습) 2020년 상품별 매출액을 구하시오
- (힌트) 2020년 판매량과 가격을 곱하면 된다

In [91]:
# (풀이)



## 특정 조건 값 계산하기

In [72]:
# 2020년 미국 수출 총액
df[df.region == 'US']['2020'].sum()

319

In [73]:
# 2020년 항공 수출 평균액
df[df.logistic == 'air']['2020'].mean()

71.5

## (연습) 2020년 전기차의 항공 수출 건수를 구하시오


In [92]:
# (풀이)



# 데이터프레임 분해하기
- groupby
- 특정 조건에 맞는 그룹을 테이블로 새로 만들고 다양하게 원하는 조작/계산을 한다

In [75]:
# 품목별 2020 매출 합계 보기
df.groupby('item')['2020'].sum().to_frame()

2020
item      
Refr   297
TV     266
eCar   300

- 품목별로 2019, 2020 각각에 대해 수출 합계, 건수, 평균치 보기
 - agg(aggregate)를 사용하여 여러개의 함수 이름을 나열한다 (임의로 정의한 함수를 사용할 수도 있다)
 - size는 항목의 갯수를 알려준다

In [76]:
df.groupby('item')[['2019','2020']].agg(['sum', 'size','mean'])

2019             2020            
      sum size   mean  sum size   mean
item                                  
Refr  143    4  35.75  297    4  74.25
TV    138    4  34.50  266    4  66.50
eCar  159    4  39.75  300    4  75.00

In [77]:
# 품목별, 지역별로 2020년 판매 평균치 보기
df.groupby(['item', 'region'])['2020'].mean().to_frame()

2020
item region       
Refr China   111.0
     US       37.5
TV   China    72.5
     US       60.5
eCar China    88.5
     US       61.5

In [78]:
# 품목별, 지역별, 운송별로 2019, 2020년 판매 합계 보기
df.groupby(['item', 'region','logistic'])[['2019','2020']].sum()

2019  2020
item region logistic            
Refr China  air         56    42
            ship        29   180
     US     air         23    30
            ship        35    45
TV   China  air         56    89
            ship        45    56
     US     air         13    87
            ship        24    34
eCar China  air         65    79
            ship        48    98
     US     air         27   102
            ship        19    21

## 조건에 맞는 그룹을 각각 얻기

- groupby()의 실행 결과는 그룹명과 데이터프레임 두 가지를 리턴한다
- 특정 그룹만 얻으려면 get_group()를 사용한다

In [79]:
# item 기준으로 그룹을 나누고 그중 TV에 해당하는 내용만 별도의 데이터프레임으로 얻는다
df.groupby(['item']).get_group('TV')

item logistic  2019  2020 region
2   TV     ship    24    34     US
3   TV      air    13    87     US
8   TV     ship    45    56  China
9   TV      air    56    89  China

In [80]:
# item 기준으로 나눈 그룹을 각각 별도의 csv 파일로 저장하는 예
for item, group in df.groupby('item'):
    group.to_csv(item + '.csv')

In [81]:
# TV.csv, Refr.csv, eCar.csv 파일이 생성됨
!ls *.csv
# dir *.csv # 윈도우에서 보기

A.csv                    creditcard.csv           simple_merged_ID.csv
B.csv                    customer.csv             simple_merged_ID2.csv
C.csv                    eCar.csv                 stores.csv
Life Expectancy Data.csv energy.csv               test.csv
PM_train_pre.csv         gold_price.csv           test_1.csv
Refr.csv                 merged_ID.csv            test_2.csv
TV.csv                   movies.csv               test_3.csv
big.csv                  ratings.csv
cities.csv               set_angle_0402.csv


## 조건에 맞는 그룹 찾기

In [82]:
# 품목별 2020년 판매 평균값
df.groupby('item')['2020'].size()

item
Refr    4
TV      4
eCar    4
Name: 2020, dtype: int64

## filter

In [83]:
# 2020년 품목별 평균이 70을 넘는 "상품"들의 샘플을 모두 찾기 (여기서는 eCar와 Refr 두 상품이다)
# filter를 사용하면 편리하다
# filter 수행 결과는 True/False를 얻으며 해당 샘플을 추출한다
df.groupby('item').filter(
    lambda x : x['2020'].mean() > 70 
    )

item logistic  2019  2020 region
0   eCar     ship    19    21     US
1   eCar      air    27   102     US
4   Refr     ship    35    45     US
5   Refr      air    23    30     US
6   Refr     ship    29   180  China
7   Refr      air    56    42  China
10  eCar     ship    48    98  China
11  eCar      air    65    79  China

- (주의!) 아래는 2020년 매출액이 각 "샘플별로" 70보다 큰 경우를 찾는다 (위의 결과와 다르다)

In [84]:
df[df["2020"] > 70]

item logistic  2019  2020 region
1   eCar      air    27   102     US
3     TV      air    13    87     US
6   Refr     ship    29   180  China
9     TV      air    56    89  China
10  eCar     ship    48    98  China
11  eCar      air    65    79  China

# pivot_table
- 조건에 맞는 수치 요약 테이블을 생성한다
- groupby는 그룹을 생성하는 중간 과정이 있으나 피봇 페이블은 수치 요약 결과만 보여준다

In [86]:
# 상품별 2020년 총 매출액
df.pivot_table(index=['item'], values=['2020'], aggfunc='sum')

2020
item      
Refr   297
TV     266
eCar   300

In [87]:
# 상품, 지역별 2020년 매출 합계 보기
df.pivot_table(index=['item', 'region'], values=['2020'],aggfunc=['sum'])

sum
            2020
item region     
Refr China   222
     US       75
TV   China   145
     US      121
eCar China   177
     US      123

## (연습) 상품, 지역별 2020년 매출 합계와 평균 보기

In [93]:
# (풀이)



In [87]:
# column을 지정하면 컬럼을 세분화하여 보여준다
# 위의 결과를 운송 수단별로 더 세분화하여 보기
df.pivot_table(index=['item', 'region'], values=['2020'],aggfunc=['sum','mean'],
              columns=['logistic'])

sum      mean     
            2020      2020     
logistic     air ship  air ship
item region                    
Refr China    42  180   42  180
     US       30   45   30   45
TV   China    89   56   89   56
     US       87   34   87   34
eCar China    79   98   79   98
     US      102   21  102   21

# 정답

## (연습) 좌우 데이터프레임 인덱스 중 공통부분만을 대상으로 붙이시오
- (힌트) 'inner" 옵션을 사용한다

In [54]:
# 좌우 데이터프레임 인덱스 중 공통부분을 대상으로 붙이는 경우
df1.join(df4, how='inner')

population  temp  sale_1  sale_2
city                                  
대전           154    13      11      12
광주           150    15      21      22

## (연습) 두 데이터프레임의 컬럼 city를 기준으로 모든 행을 합치시오
- (힌트) outer 옵션을 사용한다

In [61]:
# 전체 데이터프레임의 컬럼 city를 기준으로 합치는 경우
df1.merge(df4, on='city', how='outer')

city  population  temp  sale_1  sale_2
0   서울       990.0  13.0     NaN     NaN
1   부산       350.0  16.0     NaN     NaN
2   대구       250.0  14.0     NaN     NaN
3   대전       154.0  13.0    11.0    12.0
4   광주       150.0  15.0    21.0    22.0
5   인천         NaN   NaN    31.0    32.0
6   울산         NaN   NaN    41.0    42.0

## (연습) 두 데이터프레임의 컬럼 city를 기준으로 모든 행을 합치시오
- (힌트) outer 옵션을 사용한다

In [61]:
# 전체 데이터프레임의 컬럼 city를 기준으로 합치는 경우
df1.merge(df4, on='city', how='outer')

city  population  temp  sale_1  sale_2
0   서울       990.0  13.0     NaN     NaN
1   부산       350.0  16.0     NaN     NaN
2   대구       250.0  14.0     NaN     NaN
3   대전       154.0  13.0    11.0    12.0
4   광주       150.0  15.0    21.0    22.0
5   인천         NaN   NaN    31.0    32.0
6   울산         NaN   NaN    41.0    42.0

## (연습) 2020년 전기차의 항공 수출 건수를 구하시오


In [74]:
df[(df.item=='eCar')&(df.logistic == 'air')]['2020'].count()

2

## (연습) 상품, 지역별 2020년 매출 합계와 평균 보기

In [88]:
df.pivot_table(index=['item', 'region'], values=['2020'],aggfunc=['sum','mean'])

sum   mean
            2020   2020
item region            
Refr China   222  111.0
     US       75   37.5
TV   China   145   72.5
     US      121   60.5
eCar China   177   88.5
     US      123   61.5